<a href="https://colab.research.google.com/github/alexjercan/unsupervised-segmentation/blob/master/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install matplotlib==3.3.3 albumentations==0.5.2

     |████████████████████████████████| 11.6MB 242kB/s 
     |████████████████████████████████| 81kB 13.4MB/s 
     |████████████████████████████████| 38.2MB 78kB/s 
     |████████████████████████████████| 952kB 43.0MB/s 
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [1]:

!git clone https://github.com/alexjercan/unsupervised-segmentation.git
%cd unsupervised-segmentation

import torch
from IPython.display import clear_output

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
# Download model
torch.hub.download_url_to_file('https://github.com/alexjercan/unsupervised-segmentation/releases/download/v1.0/normal.pth', 'normal.pth')

In [2]:
# Download dataset
torch.hub.download_url_to_file('https://github.com/alexjercan/unsupervised-segmentation/releases/download/v1.0/bdataset_scene.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../ && rm tmp.zip

In [3]:
!bash ./get_bdataset.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1441      0 --:--:-- --:--:-- --:--:--  1441
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 7494M    0 7494M    0     0  42.7M      0 --:--:--  0:02:55 --:--:-- 26.4M
Completed in 577 seconds


In [9]:
!git pull
!python dataset.py
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
!python model.py
end.record()
torch.cuda.synchronize()
print(start.elapsed_time(end))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
model ok
6129.1357421875


In [1]:
%cd unsupervised-segmentation/

/content/unsupervised-segmentation


In [2]:
import os
import re
import cv2
import torch
import torch.optim
import albumentations as A
import my_albumentations as M

from datetime import datetime as dt
from albumentations.pytorch.transforms import ToTensorV2
from config import DEVICE
from model import Model, LossFunction
from general import init_weights, load_checkpoint, save_checkpoint
from dataset import create_dataloader, create_dataloader_nyuv2, LoadImages
from metrics import MetricFunction, print_single_error, MetricFunctionNYUv2
from detect import generatePredictions
from train import train_one_epoch
from test import run_test, run_test_nyuv2
from util import save_predictions, plot_predictions

IMAGE_SIZE = 256
DATASET_ROOT = "../bdataset"
TRAIN_JSON_PATH = "train.json"
TEST_JSON_PATH = "test.json"
IMAGES = [
          {"image": "data/0000.png", "depth": "data/0000.exr", "output": "data/out/0000.png"},
          {"image": "data/0001.png", "depth": "data/0001.exr", "output": "data/out/0001.png"},
          {"image": "data/0002.png", "depth": "data/0002.exr", "output": "data/out/0002.png"}
]
BATCH_SIZE = 8
WORKERS = 8

LEARNING_RATE = 1e-5
BETAS = [0.9, 0.999]
EPS = 1e-8
WEIGHT_DECAY = 1e-4

MILESTONES = [10]
GAMMA = 0.1

NUM_EPOCHS = 20
OUT_PATH ="./runs"
LOAD_TRAIN_MODEL = False
LOAD_TEST_MODEL = False
CHECKPOINT_TRAIN_FILE = "normal.pth"
CHECKPOINT_TEST_FILE = "normal.pth"

torch.backends.cudnn.benchmark = True

In [3]:
train_transform = A.Compose(
    [
        M.MyRandomResizedCrop(width=IMAGE_SIZE, height=IMAGE_SIZE),
        M.MyHorizontalFlip(p=0.5),
        M.MyVerticalFlip(p=0.1),
        A.OneOf([
            A.MotionBlur(p=0.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.OneOf([
            M.MyOpticalDistortion(p=0.3),
            M.MyGridDistortion(p=0.1),
        ], p=0.2),
        A.OneOf([
            A.IAASharpen(),
            A.IAAEmboss(),
            A.RandomBrightnessContrast(),
        ], p=0.3),
        A.Normalize(),
        M.MyToTensorV2(),
    ],
    additional_targets={
        'normal': 'normal',
        'depth': 'depth',
    }
)

test_transform = A.Compose(
    [
        A.Normalize(),
        M.MyToTensorV2(),
    ],
    additional_targets={
        'normal': 'normal',
        'depth': 'depth',
    }
)

detect_transform = A.Compose(
    [
        M.MyLongestMaxSize(max_size=IMAGE_SIZE),
        M.MyPadIfNeeded(min_height=IMAGE_SIZE, min_width=IMAGE_SIZE, border_mode=cv2.BORDER_CONSTANT, value=0),
        A.Normalize(),
        M.MyToTensorV2(),
    ],
    additional_targets={
        'depth' : 'depth',
    }
)

_, train_dataloader = create_dataloader(DATASET_ROOT, TRAIN_JSON_PATH, 
                                        batch_size=BATCH_SIZE, transform=train_transform, 
                                        workers=WORKERS, pin_memory=True, shuffle=True)

_, test_dataloader = create_dataloader(DATASET_ROOT, TEST_JSON_PATH,
                                       batch_size=BATCH_SIZE, transform=test_transform,
                                       workers=WORKERS, pin_memory=True, shuffle=False)

In [ ]:
model = Model(num_classes=10, num_layers=3)
model.apply(init_weights)
solver = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                          lr=LEARNING_RATE, betas=BETAS, 
                          eps=EPS, weight_decay=WEIGHT_DECAY)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(solver, milestones=MILESTONES, gamma=GAMMA)
model = model.to(DEVICE)
loss_fn = LossFunction()

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

print(get_n_params(model))

75530090


In [ ]:
epoch_idx = 0
if LOAD_TRAIN_MODEL:
    epoch_idx, model = load_checkpoint(model, CHECKPOINT_TRAIN_FILE, DEVICE)

model.train()
for epoch_idx in range(epoch_idx, NUM_EPOCHS):
    metric_fn = MetricFunction(BATCH_SIZE)
    train_one_epoch(model, train_dataloader, loss_fn, metric_fn, solver, epoch_idx)
    print_single_error(epoch_idx, loss_fn.show(), metric_fn.show())
    lr_scheduler.step()

In [ ]:
if LOAD_TEST_MODEL:
    epoch_idx, model = load_checkpoint(model, CHECKPOINT_TEST_FILE, DEVICE)

model.eval()
metric_fn = MetricFunction(BATCH_SIZE)
run_test(model, test_dataloader, loss_fn, metric_fn)
print_single_error(epoch_idx, loss_fn.show(), metric_fn.show())

In [ ]:
# if LOAD_TEST_MODEL:
# epoch_idx, model = load_checkpoint(model, CHECKPOINT_TEST_FILE, DEVICE)

model.eval()
images = LoadImages(IMAGES, transform=detect_transform)
for img, predictions, depths, path in generatePredictions_fcn(model, images):
    plot_predictions([img], predictions, depths, [path])
    save_predictions([img], predictions, depths, [path])

In [ ]:
output_dir = os.path.join(OUT_PATH, re.sub("[^0-9a-zA-Z]+", "-", dt.now().isoformat()))

save_checkpoint(epoch_idx, model, output_dir)

NYU v2 Dataset

In [ ]:
_, test_dataloader_nyuv2 = create_dataloader_nyuv2(batch_size=BATCH_SIZE, transform=test_transform, workers=WORKERS, pin_memory=True, shuffle=False)

In [ ]:
if LOAD_TEST_MODEL:
    epoch_idx, model = load_checkpoint(model, CHECKPOINT_TEST_FILE, DEVICE)

model.eval()
metric_fn = MetricFunctionNYUv2(BATCH_SIZE)
run_test_nyuv2(model, test_dataloader_nyuv2, loss_fn, metric_fn)
print_single_error(epoch_idx, loss_fn.show(), metric_fn.show())

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:128: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  img = torch.from_numpy(np.array(pic, np.int32, copy=False))
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:128: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable befo

eval_avg_error
Epoch: 20, loss=(total:3.4694 c:0.8977 s:2.5717)
======SEGMENTATION========
IOU=0.0397	P=0.1985	R=0.0016	F1=0.0031




OG Model kanezaki github

In [ ]:
from original import OgModel, OgLossFunction, og_train_one_epoch, og_run_test, og_run_test_nyuv2

In [ ]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
!python original.py
end.record()
torch.cuda.synchronize()
print(start.elapsed_time(end))

model ok
5622.5693359375


In [ ]:
model = OgModel(num_classes=10)
model.apply(init_weights)
solver = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                         lr=LEARNING_RATE, momentum=0.9,
                         dampening=0.1, weight_decay=WEIGHT_DECAY)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(solver, milestones=MILESTONES, gamma=GAMMA)
model = model.to(DEVICE)
loss_fn = OgLossFunction()

In [ ]:
epoch_idx = 0
if LOAD_TRAIN_MODEL:
    epoch_idx, model = load_checkpoint(model, CHECKPOINT_TRAIN_FILE, DEVICE)

model.train()
for epoch_idx in range(epoch_idx, NUM_EPOCHS):
    metric_fn = MetricFunction(BATCH_SIZE)
    og_train_one_epoch(model, train_dataloader, loss_fn, metric_fn, solver, epoch_idx)
    print_single_error(epoch_idx, loss_fn.show(), metric_fn.show())
    lr_scheduler.step()

In [ ]:
if LOAD_TEST_MODEL:
    epoch_idx, model = load_checkpoint(model, CHECKPOINT_TEST_FILE, DEVICE)

model.eval()
metric_fn = MetricFunction(BATCH_SIZE)
og_run_test(model, test_dataloader, loss_fn, metric_fn)
print_single_error(epoch_idx, loss_fn.show(), metric_fn.show())

In [ ]:
if LOAD_TEST_MODEL:
    epoch_idx, model = load_checkpoint(model, CHECKPOINT_TEST_FILE, DEVICE)

model.eval()
metric_fn = MetricFunctionNYUv2(BATCH_SIZE)
og_run_test_nyuv2(model, test_dataloader_nyuv2, loss_fn, metric_fn)
print_single_error(epoch_idx, loss_fn.show(), metric_fn.show())

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:128: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  img = torch.from_numpy(np.array(pic, np.int32, copy=False))
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:128: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable befo

eval_avg_error
Epoch: 20, loss=(total:2.9240 s:1.0791 c:1.8449)
======SEGMENTATION========
IOU=0.0396	P=0.1761	R=0.0014	F1=0.0027




In [ ]:
from general import tensors_to_device, generate_layers
from tqdm import tqdm
from torchvision.models.segmentation import fcn_resnet50
from torchvision.models.segmentation.fcn import FCNHead
from detect import generatePredictions_fcn
num_layers = 3

In [ ]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
    def runmodel(model, imgs, depths):
        layers = generate_layers(imgs, depths, num_layers)
        x = [model(x)['out'] for x in layers]
        return torch.stack(x, dim=-1)

    loop = tqdm(loader, position=0, leave=True)
    for batch_idx, tensors in enumerate(loop):
        imgs, normals, depths = tensors_to_device(tensors, DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = runmodel(model, imgs, depths)
            loss = loss_fn(predictions, (normals, depths))

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())
    loop.close()

In [ ]:
model = fcn_resnet50(pretrained=True, num_classes=21)
# set_parameter_requires_grad(model)
model.classifier = FCNHead(2048, channels=14)
model = model.to(DEVICE)

epoch, model = load_checkpoint(model, "checkpoint-epoch-057.pth", DEVICE)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
scaler = torch.cuda.amp.GradScaler()
loss_fn = LossFunction()

In [ ]:
model.train()
for epoch_idx in range(100):
    train_fn(train_dataloader, model, optimizer, loss_fn, scaler)

In [26]:
output_dir = os.path.join(OUT_PATH, re.sub("[^0-9a-zA-Z]+", "-", dt.now().isoformat()))

save_checkpoint(epoch_idx, model, output_dir)

Visual Classification

In [4]:
from train import train_one_epoch_fg
from torchvision.models.resnet import resnet50
from general import set_parameter_requires_grad, tensors_to_device
import torch.nn as nn
from model import FGLossFunction
from metrics import FGMetricFunction
from dataset import create_dataloader_fg
from tqdm import tqdm

In [5]:
train_transform = A.Compose(
    [
        M.MyRandomResizedCrop(width=IMAGE_SIZE, height=IMAGE_SIZE),
        M.MyHorizontalFlip(p=0.5),
        M.MyVerticalFlip(p=0.1),
        A.OneOf([
            A.MotionBlur(p=0.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.OneOf([
            M.MyOpticalDistortion(p=0.3),
            M.MyGridDistortion(p=0.1),
        ], p=0.2),
        A.OneOf([
            A.IAASharpen(),
            A.IAAEmboss(),
            A.RandomBrightnessContrast(),
        ], p=0.3),
        A.Normalize(),
        M.MyToTensorV2(),
    ],
    additional_targets={
        'depth': 'depth',
    }
)

test_transform = A.Compose(
    [
        A.Normalize(),
        M.MyToTensorV2(),
    ],
    additional_targets={
        'depth': 'depth',
    }
)

detect_transform = A.Compose(
    [
        M.MyLongestMaxSize(max_size=IMAGE_SIZE),
        M.MyPadIfNeeded(min_height=IMAGE_SIZE, min_width=IMAGE_SIZE, border_mode=cv2.BORDER_CONSTANT, value=0),
        A.Normalize(),
        M.MyToTensorV2(),
    ],
    additional_targets={
        'depth' : 'depth',
    }
)

_, train_dataloader = create_dataloader_fg(DATASET_ROOT, TRAIN_JSON_PATH, 
                                        batch_size=BATCH_SIZE, transform=train_transform, 
                                        workers=WORKERS, pin_memory=True, shuffle=True)

_, test_dataloader = create_dataloader_fg(DATASET_ROOT, TEST_JSON_PATH,
                                       batch_size=BATCH_SIZE, transform=test_transform,
                                       workers=WORKERS, pin_memory=True, shuffle=False)

In [6]:
model = resnet50(pretrained=True)
set_parameter_requires_grad(model)
model.fc = nn.Linear(512 * 4, 30)
model = model.to(DEVICE)

In [7]:
solver = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                          lr=1e-3, betas=BETAS, 
                          eps=EPS, weight_decay=WEIGHT_DECAY)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(solver, milestones=MILESTONES, gamma=GAMMA)
loss_fn = FGLossFunction()

In [8]:
# epoch_idx = 0
# if LOAD_TRAIN_MODEL:
# epoch_idx, model = load_checkpoint(model, CHECKPOINT_TRAIN_FILE, DEVICE)

model.train()
for epoch_idx in range(NUM_EPOCHS):
    metric_fn = FGMetricFunction(BATCH_SIZE)
    train_one_epoch_fg(model, train_dataloader, loss_fn, metric_fn, solver, epoch_idx)
    print_single_error(epoch_idx, loss_fn.show(), metric_fn.show())
    lr_scheduler.step()

  0%|          | 0/2008 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 0/2008 [00:00<?, ?it/s]

eval_avg_error
Epoch: 0, loss=(total:1.9447)
======SEGMENTATION========\ACC=0.2714




  0%|          | 0/2008 [00:00<?, ?it/s]

eval_avg_error
Epoch: 1, loss=(total:1.3468)
======SEGMENTATION========\ACC=0.3744




  0%|          | 0/2008 [00:00<?, ?it/s]

...

eval_avg_error
Epoch: 17, loss=(total:0.8449)
======SEGMENTATION========\ACC=0.5581




  0%|          | 0/2008 [00:00<?, ?it/s]

eval_avg_error
Epoch: 18, loss=(total:2.1857)
======SEGMENTATION========\ACC=0.5519




100%|██████████| 2008/2008 [02:50<00:00, 11.77it/s, epoch=19, loss=(total:2.4873)]

eval_avg_error
Epoch: 19, loss=(total:2.4873)
======SEGMENTATION========\ACC=0.5594




In [35]:
model.train()
for epoch_idx in range(NUM_EPOCHS):
    metric_fn = FGMetricFunction(BATCH_SIZE)
    train_one_epoch_fg_og(model, train_dataloader, loss_fn, metric_fn, solver, epoch_idx)
    print_single_error(epoch_idx, loss_fn.show(), metric_fn.show())
    lr_scheduler.step()

  0%|          | 0/2008 [00:00<?, ?it/s]

eval_avg_error
Epoch: 0, loss=(total:0.6900)
======SEGMENTATION========\ACC=0.5113




  0%|          | 0/2008 [00:00<?, ?it/s]

eval_avg_error
Epoch: 1, loss=(total:2.4825)
======SEGMENTATION========\ACC=0.5383




  0%|          | 0/2008 [00:00<?, ?it/s]

...

eval_avg_error
Epoch: 10, loss=(total:0.7943)
======SEGMENTATION========\ACC=0.6739




  0%|          | 0/2008 [00:00<?, ?it/s]

eval_avg_error
Epoch: 11, loss=(total:1.0338)
======SEGMENTATION========\ACC=0.6639




  6%|▌         | 125/2008 [00:07<01:56, 16.20it/s, epoch=12, loss=(total:1.0669)]

KeyboardInterrupt: ignored

test fg fcn

In [5]:
from train import train_one_epoch_fg_fcn
from torchvision.models.segmentation import fcn_resnet50
from torchvision.models.segmentation.fcn import FCNHead
from general import set_parameter_requires_grad, tensors_to_device
import torch.nn as nn
from model import FGFCNLossFunction
from metrics import FCNFGMetricFunction
from dataset import create_dataloader_fg
from tqdm import tqdm

In [6]:
train_transform = A.Compose(
    [
        M.MyRandomResizedCrop(width=IMAGE_SIZE, height=IMAGE_SIZE),
        M.MyHorizontalFlip(p=0.5),
        M.MyVerticalFlip(p=0.1),
        A.OneOf([
            A.MotionBlur(p=0.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.OneOf([
            M.MyOpticalDistortion(p=0.3),
            M.MyGridDistortion(p=0.1),
        ], p=0.2),
        A.OneOf([
            A.IAASharpen(),
            A.IAAEmboss(),
            A.RandomBrightnessContrast(),
        ], p=0.3),
        A.Normalize(),
        M.MyToTensorV2(),
    ],
    additional_targets={
        'depth': 'depth',
    }
)

test_transform = A.Compose(
    [
        A.Normalize(),
        M.MyToTensorV2(),
    ],
    additional_targets={
        'depth': 'depth',
    }
)

detect_transform = A.Compose(
    [
        M.MyLongestMaxSize(max_size=IMAGE_SIZE),
        M.MyPadIfNeeded(min_height=IMAGE_SIZE, min_width=IMAGE_SIZE, border_mode=cv2.BORDER_CONSTANT, value=0),
        A.Normalize(),
        M.MyToTensorV2(),
    ],
    additional_targets={
        'depth' : 'depth',
    }
)

_, train_dataloader = create_dataloader_fg(DATASET_ROOT, TRAIN_JSON_PATH, 
                                        batch_size=BATCH_SIZE, transform=train_transform, 
                                        workers=WORKERS, pin_memory=True, shuffle=True)

_, test_dataloader = create_dataloader_fg(DATASET_ROOT, TEST_JSON_PATH,
                                       batch_size=BATCH_SIZE, transform=test_transform,
                                       workers=WORKERS, pin_memory=True, shuffle=False)

In [7]:
model = fcn_resnet50(pretrained=True, num_classes=21)
set_parameter_requires_grad(model)
model.classifier = FCNHead(2048, channels=31)
model = model.to(DEVICE)

Downloading: "https://download.pytorch.org/models/fcn_resnet50_coco-1167a1af.pth" to /root/.cache/torch/hub/checkpoints/fcn_resnet50_coco-1167a1af.pth


In [8]:
solver = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                          lr=1e-3, betas=BETAS, 
                          eps=EPS, weight_decay=WEIGHT_DECAY)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(solver, milestones=MILESTONES, gamma=GAMMA)
loss_fn = FGFCNLossFunction()

In [ ]:
# epoch_idx = 0
# if LOAD_TRAIN_MODEL:
# epoch_idx, model = load_checkpoint(model, CHECKPOINT_TRAIN_FILE, DEVICE)

model.train()
for epoch_idx in range(NUM_EPOCHS):
    metric_fn = FCNFGMetricFunction(BATCH_SIZE)
    train_one_epoch_fg_fcn(model, train_dataloader, loss_fn, metric_fn, solver, epoch_idx)
    print_single_error(epoch_idx, loss_fn.show(), metric_fn.show())
    lr_scheduler.step()

  0%|          | 0/2008 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 0/2008 [00:00<?, ?it/s]

eval_avg_error
Epoch: 0, loss=(total:0.4072)
======SEGMENTATION========
IOU=0.5109	P=0.0097	R=0.0012	F1=0.0010




  0%|          | 0/2008 [00:00<?, ?it/s]

eval_avg_error
Epoch: 1, loss=(total:0.1997)
======SEGMENTATION========
IOU=0.4839	P=0.0086	R=0.0004	F1=0.0006




  0%|          | 0/2008 [00:00<?, ?it/s]

...

  0%|          | 0/2008 [00:00<?, ?it/s]

eval_avg_error
Epoch: 3, loss=(total:0.8615)
======SEGMENTATION========
IOU=0.4780	P=0.0070	R=0.0001	F1=0.0002




  0%|          | 0/2008 [00:00<?, ?it/s]

eval_avg_error
Epoch: 4, loss=(total:1.2077)
======SEGMENTATION========
IOU=0.4758	P=0.0075	R=0.0001	F1=0.0002




 14%|█▍        | 280/2008 [00:50<05:10,  5.56it/s, epoch=5, loss=(total:0.6628)]